In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test_labels.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train-processed-seqlen128.csv
/kaggle/input/jigsaw-mlm-finetuned-xlm-r-large/config.json
/kaggle/input/jig

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.9/778.9 kB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.1/436.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [4]:
MAX_LEN = 192 
DROPOUT = 0.5 # use aggressive dropout
BATCH_SIZE = 16 # per TPU core
TOTAL_STEPS_STAGE1 = 2000
VALIDATE_EVERY_STAGE1 = 200
TOTAL_STEPS_STAGE2 = 200
VALIDATE_EVERY_STAGE2 = 10

### Different learning rate for transformer and head ###
LR_TRANSFORMER = 5e-6
LR_HEAD = 1e-3

PRETRAINED_TOKENIZER=  'jplu/tf-xlm-roberta-large'
PRETRAINED_MODEL = '/kaggle/input/jigsaw-mlm-finetuned-xlm-r-large'
D = '/kaggle/input/jigsaw-multilingual-toxic-comment-classification/'
D_TRANS = '/kaggle/input/jigsaw-train-multilingual-coments-google-api/'


import os
import numpy as np
import pandas as pd
import sentencepiece
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import transformers
from transformers import TFRobertaModel, AutoTokenizer
import logging
# no extensive logging 
logging.getLogger().setLevel(logging.NOTSET)

AUTO = tf.data.experimental.AUTOTUNE

D0907 19:43:14.959937278      14 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0907 19:43:14.959963400      14 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0907 19:43:14.959967013      14 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0907 19:43:14.959969930      14 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0907 19:43:14.959972431      14 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0907 19:43:14.959975468      14 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0907 19:43:14.959977967      14 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0907 19:43:14.

2.12.0


/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from tqdm import tqdm

In [6]:
def connect_to_TPU():
    """Detect hardware, return appropriate distribution strategy"""
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.TPUStrategy(tpu)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    global_batch_size = BATCH_SIZE * strategy.num_replicas_in_sync

    return tpu, strategy, global_batch_size


tpu, strategy, global_batch_size = connect_to_TPU()
print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/

In [7]:
def load_jigsaw_trans(langs=['tr','it','es','ru','fr','pt'], 
                      columns=['comment_text', 'toxic']):
    train_6langs=[]
    for i in range(len(langs)):
        fn = D_TRANS+'jigsaw-toxic-comment-train-google-%s-cleaned.csv'%langs[i]
        train_6langs.append(downsample(pd.read_csv(fn)[columns]))

    return train_6langs

def downsample(df):
    """Subsample the train dataframe to 50%-50%"""
    ds_df= pd.concat([
        df.query('toxic==1'),
        df.query('toxic==0').sample(sum(df.toxic))
    ])
    
    return ds_df
    

train_df = pd.concat(load_jigsaw_trans()) 
val_df = pd.read_csv(D+'validation.csv')
test_df = pd.read_csv(D+'test.csv')
sub_df = pd.read_csv(D+'sample_submission.csv')

In [8]:
%%time

def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])
    

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_TOKENIZER)
X_train = regular_encode(train_df.comment_text.values.tolist(), tokenizer, maxlen=MAX_LEN)
X_val = regular_encode(val_df.comment_text.values.tolist(), tokenizer, maxlen=MAX_LEN)
X_test = regular_encode(test_df.content.values.tolist(), tokenizer, maxlen=MAX_LEN)

y_train = train_df.toxic.values.reshape(-1,1)
y_val = val_df.toxic.values.reshape(-1,1)

/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad t

CPU times: user 9min 58s, sys: 23.6 s, total: 10min 22s
Wall time: 42.6 s


In [9]:
def create_dist_dataset(X, y=None, training=False):
    dataset = tf.data.Dataset.from_tensor_slices(X)

    ### Add y if present ###
    if y is not None:
        dataset_y = tf.data.Dataset.from_tensor_slices(y)
        dataset = tf.data.Dataset.zip((dataset, dataset_y))
        
    ### Repeat if training ###
    if training:
        dataset = dataset.shuffle(len(X)).repeat()

    dataset = dataset.batch(global_batch_size).prefetch(AUTO)

    ### make it distributed  ###
    dist_dataset = strategy.experimental_distribute_dataset(dataset)

    return dist_dataset
    
    
train_dist_dataset = create_dist_dataset(X_train, y_train, True)
val_dist_dataset   = create_dist_dataset(X_val)
test_dist_dataset  = create_dist_dataset(X_test)

In [10]:
%%time
def create_model_and_optimizer():
    with strategy.scope():
        transformer_layer = TFRobertaModel.from_pretrained(PRETRAINED_MODEL)  
#         transformer_layer = TransformerLayer(base_transformer)
        model = build_model(transformer_layer)
        optimizer_transformer = Adam(learning_rate=LR_TRANSFORMER)
        optimizer_head = Adam(learning_rate=LR_HEAD)
    return model, optimizer_transformer, optimizer_head


def build_model(transformer):
    inp = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_word_ids")
    # Huggingface transformers have multiple outputs, embeddings are the first one
    # let's slice out the first position, the paper says its not worse than pooling
    
    x = transformer(inp)[0][:, 0, :]  
    x = Dropout(DROPOUT)(x)
    ### note, adding the name to later identify these weights for different LR
    out = Dense(1, activation='sigmoid', name='custom_head')(x)
    model = Model(inputs=[inp], outputs=[out])
    
    return model


model, optimizer_transformer, optimizer_head = create_model_and_optimizer()

model.summary()

You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some layers from the model checkpoint at /kaggle/input/jigsaw-mlm-finetuned-xlm-r-large were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at /kaggle/input/jigsaw-mlm-finetuned-xlm-r-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFR

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 192)]            0         
                                                                 
 tf_roberta_model (TFRoberta  TFBaseModelOutputWithPoo  559890432
 Model)                      lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             192, 1024),                         
                              pooler_output=(None, 10            
                             24),                                
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                             

In [11]:
import wandb


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: W&B API key is config

True

In [12]:
def define_losses_and_metrics():
    with strategy.scope():
        loss_object = tf.keras.losses.BinaryCrossentropy(
            reduction=tf.keras.losses.Reduction.NONE, from_logits=False)

        def compute_loss(labels, predictions):
            per_example_loss = loss_object(labels, predictions)
            loss = tf.nn.compute_average_loss(
                per_example_loss, global_batch_size = global_batch_size)
            return loss

        # AUC and Loss Metrics
        train_auc_metric = tf.keras.metrics.AUC(name='training_AUC')
        val_auc_metric = tf.keras.metrics.AUC(name='validation_AUC')
        train_loss_metric = tf.keras.metrics.Mean(name='training_loss')  # Tracking loss
        val_loss_metric = tf.keras.metrics.Mean(name='validation_loss')  # Validation loss

    return compute_loss, train_auc_metric, val_auc_metric, train_loss_metric, val_loss_metric

# Initialize wandb before training
wandb.init(project="roberta", name="1") # Set project name and experiment name

def train(train_dist_dataset, val_dist_dataset=None, y_val=None,
          total_steps=2000, validate_every=200):
    best_weights, history = None, []
    step = 0

    ### Training loop ###
    for tensor in train_dist_dataset:
        distributed_train_step(tensor)
        step += 1

        if (step % validate_every == 0):   
            ### Train Metrics ###
            train_auc = train_auc_metric.result().numpy()
            train_loss = train_loss_metric.result().numpy()

            print("Step %d, train AUC: %.5f, train Loss: %.5f" % (step, train_auc, train_loss))

            # Wandb log for training
            wandb.log({"train_AUC": train_auc, "train_loss": train_loss, "step": step})

            ### Validation Metrics ###
            if val_dist_dataset:
                val_predictions = predict(val_dist_dataset)
                val_auc = roc_auc_score(y_val, val_predictions)
                val_loss = compute_val_loss(y_val, val_predictions)  # Custom validation loss computation

                print("Step %d, val AUC: %.5f, val Loss: %.5f" % (step, val_auc, val_loss))

                # Wandb log for validation
                wandb.log({"val_AUC": val_auc, "val_loss": val_loss, "step": step})

                history.append(val_auc)
                if history[-1] == max(history):
                    best_weights = model.get_weights()

            ### Reset metrics ###
            train_auc_metric.reset_states()
            train_loss_metric.reset_states()

        if step == total_steps:
            break

    ### Restore best weights ###
    model.set_weights(best_weights)

@tf.function
def distributed_train_step(data):
    strategy.run(train_step, args=(data,))

def train_step(inputs):
    features, labels = inputs

    ### Separate transformer and head variables
    transformer_trainable_variables = [v for v in model.trainable_variables 
                                       if (('pooler' not in v.name)  and ('custom' not in v.name))]
    head_trainable_variables = [v for v in model.trainable_variables 
                                if 'custom' in v.name]

    with tf.GradientTape(persistent=True) as tape:
        predictions = model(features, training=True)
        loss = compute_loss(labels, predictions)

    gradients_transformer = tape.gradient(loss, transformer_trainable_variables)
    gradients_head = tape.gradient(loss, head_trainable_variables)
    del tape

    optimizer_transformer.apply_gradients(zip(gradients_transformer, transformer_trainable_variables))
    optimizer_head.apply_gradients(zip(gradients_head, head_trainable_variables))

    train_auc_metric.update_state(labels, predictions)
    train_loss_metric.update_state(loss)  # Log training loss metric

def predict(dataset):  
    predictions = []
    for tensor in dataset:
        predictions.append(distributed_prediction_step(tensor))
    predictions = np.vstack(list(map(np.vstack, predictions)))
    return predictions

@tf.function
def distributed_prediction_step(data):
    predictions = strategy.run(prediction_step, args=(data,))
    return strategy.experimental_local_results(predictions)

def prediction_step(inputs):
    features = inputs
    predictions = model(features, training=False)
    return predictions

def compute_val_loss(y_val, val_predictions):
    # Custom function to calculate validation loss
    val_loss = tf.keras.losses.binary_crossentropy(y_val, val_predictions, from_logits=False)
    return tf.reduce_mean(val_loss).numpy()

# Initialize metrics
compute_loss, train_auc_metric, val_auc_metric, train_loss_metric, val_loss_metric = define_losses_and_metrics()


wandb: Currently logged in as: bishoymeseha (bishoymeseha-n-a). Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible


In [13]:
%%time
train(train_dist_dataset, val_dist_dataset, y_val,
      TOTAL_STEPS_STAGE1, VALIDATE_EVERY_STAGE1)
wandb.finish()

DEBUG:absl:`TPUStrategy.run` is called with [args: ((PerReplica:{
  0: <tf.Tensor 'data:0' shape=(16, 192) dtype=int64>,
  1: <tf.Tensor 'data_1:0' shape=(16, 192) dtype=int64>,
  2: <tf.Tensor 'data_2:0' shape=(16, 192) dtype=int64>,
  3: <tf.Tensor 'data_3:0' shape=(16, 192) dtype=int64>,
  4: <tf.Tensor 'data_4:0' shape=(16, 192) dtype=int64>,
  5: <tf.Tensor 'data_5:0' shape=(16, 192) dtype=int64>,
  6: <tf.Tensor 'data_6:0' shape=(16, 192) dtype=int64>,
  7: <tf.Tensor 'data_7:0' shape=(16, 192) dtype=int64>
}, PerReplica:{
  0: <tf.Tensor 'data_8:0' shape=(16, 1) dtype=int64>,
  1: <tf.Tensor 'data_9:0' shape=(16, 1) dtype=int64>,
  2: <tf.Tensor 'data_10:0' shape=(16, 1) dtype=int64>,
  3: <tf.Tensor 'data_11:0' shape=(16, 1) dtype=int64>,
  4: <tf.Tensor 'data_12:0' shape=(16, 1) dtype=int64>,
  5: <tf.Tensor 'data_13:0' shape=(16, 1) dtype=int64>,
  6: <tf.Tensor 'data_14:0' shape=(16, 1) dtype=int64>,
  7: <tf.Tensor 'data_15:0' shape=(16, 1) dtype=int64>
}),)] [kwargs: None]

Step 200, train AUC: 0.88737, train Loss: 0.05321


DEBUG:absl:`TPUStrategy.run` is called with [args: (PerReplica:{
  0: <tf.Tensor 'data:0' shape=(16, 192) dtype=int64>,
  1: <tf.Tensor 'data_1:0' shape=(16, 192) dtype=int64>,
  2: <tf.Tensor 'data_2:0' shape=(16, 192) dtype=int64>,
  3: <tf.Tensor 'data_3:0' shape=(16, 192) dtype=int64>,
  4: <tf.Tensor 'data_4:0' shape=(16, 192) dtype=int64>,
  5: <tf.Tensor 'data_5:0' shape=(16, 192) dtype=int64>,
  6: <tf.Tensor 'data_6:0' shape=(16, 192) dtype=int64>,
  7: <tf.Tensor 'data_7:0' shape=(16, 192) dtype=int64>
},)] [kwargs: None]
2024-09-07 19:49:29.180414: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node model/custom_head/BiasAdd/ReadVariableOp.
2024-09-07 19:49:29.418187: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node model/custom_head/BiasAdd/ReadVariableOp.
DEBUG:absl:`TPUStrategy.run` is called with [arg

Step 200, val AUC: 0.94196, val Loss: 0.31791
Step 400, train AUC: 0.96723, train Loss: 0.02861
Step 400, val AUC: 0.94416, val Loss: 0.35647
Step 600, train AUC: 0.96795, train Loss: 0.02810
Step 600, val AUC: 0.94265, val Loss: 0.36436
Step 800, train AUC: 0.97120, train Loss: 0.02667
Step 800, val AUC: 0.94390, val Loss: 0.33151
Step 1000, train AUC: 0.97227, train Loss: 0.02614
Step 1000, val AUC: 0.94408, val Loss: 0.36111
Step 1200, train AUC: 0.97377, train Loss: 0.02533
Step 1200, val AUC: 0.94245, val Loss: 0.30119
Step 1400, train AUC: 0.97413, train Loss: 0.02510
Step 1400, val AUC: 0.94591, val Loss: 0.29165
Step 1600, train AUC: 0.97434, train Loss: 0.02488
Step 1600, val AUC: 0.94476, val Loss: 0.36254
Step 1800, train AUC: 0.97619, train Loss: 0.02391
Step 1800, val AUC: 0.94656, val Loss: 0.22958
Step 2000, train AUC: 0.97630, train Loss: 0.02399
Step 2000, val AUC: 0.94270, val Loss: 0.34940


step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
train_AUC,▁▇▇███████
train_loss,█▂▂▂▂▁▁▁▁▁
val_AUC,▁▄▂▄▄▂▇▅█▂
val_loss,▆██▆█▅▄█▁▇
step,2000
train_AUC,0.9763
train_loss,0.02399
val_AUC,0.9427
val_loss,0.3494


CPU times: user 13min 30s, sys: 1min 55s, total: 15min 26s
Wall time: 15min 16s


In [14]:
X_train = regular_encode(train_df.comment_text.values.tolist(), tokenizer, maxlen=MAX_LEN)
X_val = regular_encode(val_df.comment_text.values.tolist(), tokenizer, maxlen=MAX_LEN)
X_test = regular_encode(test_df.content.values.tolist(), tokenizer, maxlen=MAX_LEN)

y_train = train_df.toxic.values.reshape(-1,1)
y_val = val_df.toxic.values.reshape(-1,1)

/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [15]:
# %%time
%pdb on

# decrease LR for second stage in the head
optimizer_head.learning_rate.assign(1e-4)

# split validation data into train test
X_train, X_val, y_train, y_val = train_test_split(X_val, y_val, test_size = 0.1)

# make a datasets
train_dist_dataset = create_dist_dataset(X_train, y_train, training=True)
val_dist_dataset = create_dist_dataset(X_val)
wandb.init(project="roberta", name="2")# train again
train(train_dist_dataset, val_dist_dataset, y_val,
      total_steps = TOTAL_STEPS_STAGE2, 
      validate_every = VALIDATE_EVERY_STAGE2)  # not validating but printing now
wandb.finish()

Automatic pdb calling has been turned ON


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step 10, train AUC: 0.94353, train Loss: 0.02634


DEBUG:absl:`TPUStrategy.run` is called with [args: (PerReplica:{
  0: <tf.Tensor 'data:0' shape=(4, 192) dtype=int64>,
  1: <tf.Tensor 'data_1:0' shape=(4, 192) dtype=int64>,
  2: <tf.Tensor 'data_2:0' shape=(4, 192) dtype=int64>,
  3: <tf.Tensor 'data_3:0' shape=(4, 192) dtype=int64>,
  4: <tf.Tensor 'data_4:0' shape=(4, 192) dtype=int64>,
  5: <tf.Tensor 'data_5:0' shape=(4, 192) dtype=int64>,
  6: <tf.Tensor 'data_6:0' shape=(4, 192) dtype=int64>,
  7: <tf.Tensor 'data_7:0' shape=(4, 192) dtype=int64>
},)] [kwargs: None]
2024-09-07 20:01:58.075096: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node model/custom_head/BiasAdd/ReadVariableOp.
2024-09-07 20:01:58.287329: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node model/custom_head/BiasAdd/ReadVariableOp.


Step 10, val AUC: 0.95789, val Loss: 0.17753
Step 20, train AUC: 0.94678, train Loss: 0.02439
Step 20, val AUC: 0.95823, val Loss: 0.17526
Step 30, train AUC: 0.92387, train Loss: 0.03073
Step 30, val AUC: 0.95889, val Loss: 0.18570
Step 40, train AUC: 0.94235, train Loss: 0.02772
Step 40, val AUC: 0.95770, val Loss: 0.17246
Step 50, train AUC: 0.95052, train Loss: 0.02336
Step 50, val AUC: 0.95857, val Loss: 0.17269
Step 60, train AUC: 0.95655, train Loss: 0.02139
Step 60, val AUC: 0.96025, val Loss: 0.17141
Step 70, train AUC: 0.95515, train Loss: 0.02322
Step 70, val AUC: 0.96247, val Loss: 0.17117
Step 80, train AUC: 0.95911, train Loss: 0.02197
Step 80, val AUC: 0.96281, val Loss: 0.16800
Step 90, train AUC: 0.95898, train Loss: 0.02330
Step 90, val AUC: 0.96370, val Loss: 0.16713
Step 100, train AUC: 0.95872, train Loss: 0.02212
Step 100, val AUC: 0.96408, val Loss: 0.16608
Step 110, train AUC: 0.96040, train Loss: 0.02157
Step 110, val AUC: 0.96414, val Loss: 0.17549
Step 120, t

step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train_AUC,▃▄▁▃▄▅▅▅▅▅▅▆▇▇▆▆▅█▇▇
train_loss,▆▅█▇▅▄▅▄▅▄▄▃▂▂▃▄▄▁▂▂
val_AUC,▁▂▂▁▂▄▆▇███▆▆█▅▇█▃▇▆
val_loss,▃▃▄▂▂▂▂▁▁▁▃▂▂█▃▂▂▅▄▄
step,200
train_AUC,0.97402
train_loss,0.01824
val_AUC,0.96206
val_loss,0.18209


In [16]:
%%time
sub_df['toxic'] = predict(test_dist_dataset)[:,0]
sub_df.to_csv('submission.csv', index=False)

DEBUG:absl:`TPUStrategy.run` is called with [args: (PerReplica:{
  0: <tf.Tensor 'data:0' shape=(9, 192) dtype=int64>,
  1: <tf.Tensor 'data_1:0' shape=(9, 192) dtype=int64>,
  2: <tf.Tensor 'data_2:0' shape=(9, 192) dtype=int64>,
  3: <tf.Tensor 'data_3:0' shape=(9, 192) dtype=int64>,
  4: <tf.Tensor 'data_4:0' shape=(9, 192) dtype=int64>,
  5: <tf.Tensor 'data_5:0' shape=(9, 192) dtype=int64>,
  6: <tf.Tensor 'data_6:0' shape=(9, 192) dtype=int64>,
  7: <tf.Tensor 'data_7:0' shape=(5, 192) dtype=int64>
},)] [kwargs: None]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, (<tf.Tensor 'data:0' shape=(9, 192) dtype=int64>,), {}]
2024-09-07 20:05:19.723509: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node model/custom_head/BiasAdd/ReadVariableOp.
2024-09-07 20:05:20.046608: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALI

CPU times: user 1min 9s, sys: 8.71 s, total: 1min 18s
Wall time: 1min 14s


In [17]:
from sklearn.metrics import roc_auc_score
ytrue = pd.read_csv(D+'test_labels.csv')['toxic']
print(roc_auc_score(ytrue, sub_df['toxic']))

0.9414304246027457
